## Aplicacion web de ML usando Flask y Render. Sin escalar data

### Librerias a utilizar

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

### Paso 1: Lectura y revision de Datos

#### Planteamiento del problema.  

El siguiente conjunto de datos, muestra los precios de las viviendas de un área de Boston, derivados del Servicio de Censos de EE.UU. Se va a realizar un modelo para predecir los precios en base a las características del Dataset, podemos iniciar tratando de contestar estas preguntas:

- ¿Existe una diferencia significativa en el valor medio de las casas delimitadas por el río Charles o no?

- ¿Existe alguna diferencia en los valores medios de las casas (MEDV) para cada proporción de unidades ocupadas por sus propietarios construidas antes de 1940 (AGE)?

- ¿Cuál es el impacto de una distancia ponderada adicional a los cinco centros de empleo de Boston sobre el valor medio de las viviendas ocupadas por sus propietarios?

#### Variables del conjunto de datos.

**- CRIM - tasa de delincuencia per cápita por ciudad**

**- ZN - proporción de suelo residencial zonificado para lotes de más de 25.000 pies cuadrados.**

**- INDUS - proporción de acres comerciales no minoristas por localidad.**

**- CHAS - variable ficticia del río Charles (1 si el tracto linda con el río; 0 en caso contrario)**

**- NOX - concentración de óxidos nítricos (partes por 10 millones)**

**- RM - número medio de habitaciones por vivienda**

**- EDAD - proporción de unidades ocupadas por sus propietarios construidas antes de 1940**

**- DIS - distancias ponderadas a cinco centros de empleo de Boston**

**- RAD - índice de accesibilidad a las autopistas radiales**

**- IMPUESTO - tasa del impuesto sobre bienes inmuebles de valor íntegro por 10.000 dólares**

**- PTRATIO - proporción alumnos-profesor por ciudad**

**- LSTAT - % más bajo de la población**

**- MEDV - valor medio de las viviendas ocupadas por sus propietarios en miles de dólares**


In [2]:
boston_df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv', usecols=lambda x: x != 'Unnamed: 0')

In [3]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,5.33,36.2


In [4]:
boston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  LSTAT    506 non-null    float64
 12  MEDV     506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


#### Características del DataSet

- Posee 506 registros.
- Tiene 12 caracteristicas tipo float64.
- No tiene valores nulos.
- No tiene registros duplicados.

In [5]:
boston_df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
LSTAT      0
MEDV       0
dtype: int64

In [6]:
boston_df.duplicated().sum()

0

In [7]:
boston_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


In [8]:
boston_df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT', 'MEDV'],
      dtype='object')

#### Seleccion de Caracteristicas

Vamos a emplear herramientas de ML para seleccionar las caracteristicas mas importantes antes de usar el modelo RandomForestRegressor

In [9]:
x_var=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT']
X = boston_df[x_var]  # Características
y = pd.DataFrame(boston_df['MEDV'])  # Variable objetivo (precio de las casas)

In [10]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFE


# Dividir el dataset reducido en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Aplicar RFE en el conjunto de entrenamiento
selector = RFE(model, n_features_to_select=4, step=1)
selector = selector.fit(X_train, y_train)

# Transformar los datos de entrenamiento y prueba
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Ajustar el modelo al conjunto de entrenamiento con las características seleccionadas
model.fit(X_train_selected, y_train)

# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test_selected)



In [11]:
# Calcular las métricas
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
#mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R-squared: {r2:.2f}")
#print(f"Mean Absolute Percentage Error: {mape:.2f}%")

Mean Squared Error: 11.11
Root Mean Squared Error: 3.33
Mean Absolute Error: 2.26
R-squared: 0.85


In [12]:
# Mostrar las características seleccionadas
selected_features = selector.support_
feature_ranking = selector.ranking_

print("Características seleccionadas y Ranking:")
for i, col in enumerate(X.columns):
    print(col,'------->',selected_features[i],'-------------->',feature_ranking[i])


Características seleccionadas y Ranking:
CRIM -------> True --------------> 1
ZN -------> False --------------> 8
INDUS -------> False --------------> 6
CHAS -------> False --------------> 9
NOX -------> False --------------> 4
RM -------> True --------------> 1
AGE -------> False --------------> 5
DIS -------> True --------------> 1
RAD -------> False --------------> 7
TAX -------> False --------------> 3
PTRATIO -------> False --------------> 2
LSTAT -------> True --------------> 1


#### Modelo final con caracteristicas seleccionadas

In [13]:
var_sel=['CRIM','RM','DIS','LSTAT']
X_sel=boston_df[var_sel]
X_sel.head()

,CRIM,RM,DIS,LSTAT
0,0.00632,6.575,4.0900,4.98
1,0.02731,6.421,4.9671,9.14
2,0.02729,7.185,4.9671,4.03
3,0.03237,6.998,6.0622,2.94
4,0.06905,7.147,6.0622,5.33


In [14]:
# Dividir el dataset reducido en conjunto de entrenamiento y prueba
X_trainf, X_testf, y_trainf, y_testf = train_test_split(X_sel, y, test_size=0.2, random_state=42)

# Crear el modelo
modelf = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenamiento Modelo con las características seleccionadas
modelf.fit(X_trainf, y_trainf)

# Predecir los valores en el conjunto de prueba
y_predf = model.predict(X_testf)


In [15]:
# Calcular las métricas
msef = mean_squared_error(y_testf, y_predf)
rmsef = np.sqrt(msef)
maef = mean_absolute_error(y_testf, y_predf)
r2f = r2_score(y_testf, y_predf)

print(f"Mean Squared Error: {msef:.2f}")
print(f"Root Mean Squared Error: {rmsef:.2f}")
print(f"Mean Absolute Error: {maef:.2f}")
print(f"R-squared: {r2f:.2f}")


Mean Squared Error: 11.11
Root Mean Squared Error: 3.33
Mean Absolute Error: 2.26
R-squared: 0.85


#### Optimizacion Modelo

- Se aplicó la técnica RandomizedSearchCV y no se pudo mejorar el modelo, se va a continuar con el modelo parametrizado con esta técnica, se denominará modelo_f.

##### RandomizedSearchCV

In [16]:
# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [10, 50, 100, 200,300],
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_depth': [None, 5, 10, 15,20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    }

# Crear un objeto RandomizedSearchCV
model_rs = RandomizedSearchCV(RandomForestRegressor(), param_distributions=param_grid, cv=10, scoring='neg_mean_squared_error', n_iter=40, verbose=1)
model_rs.fit(X_trainf, y_trainf)

Fitting 10 folds for each of 40 candidates, totalling 400 fits


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=40,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'poisson'],
                                        'max_depth': [None, 5, 10, 15, 20],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 5, 10],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'n_estimators': [10, 50, 100, 200,
                                                         300]},
                   scoring='neg_mean_squared_error', verbose=1)

In [17]:
print("Mejor puntuación:", model_rs.best_score_)
print("Mejores hiperparámetros:", model_rs.best_params_)

Mejor puntuación: -15.0637237234756
Mejores hiperparámetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 15, 'criterion': 'absolute_error'}


In [20]:
# Crear el modelo
modelo_f = RandomForestRegressor(n_estimators=100, min_samples_split= 2, min_samples_leaf=1, max_features='log2', max_depth= 15, 
                               criterion='absolute_error',random_state=42)

# Entrenamiento Modelo con las características seleccionadas
modelo_f.fit(X_trainf, y_trainf)

# Predecir los valores en el conjunto de prueba
y_predf_rs = model.predict(X_testf)


In [21]:
# Calcular las métricas
msef1 = mean_squared_error(y_testf, y_predf_rs)
rmsef1 = np.sqrt(msef1)
maef1 = mean_absolute_error(y_testf, y_predf_rs)
r2f1 = r2_score(y_testf, y_predf_rs)

print(f"Mean Squared Error: {msef1:.2f}")
print(f"Root Mean Squared Error: {rmsef1:.2f}")
print(f"Mean Absolute Error: {maef1:.2f}")
print(f"R-squared: {r2f1:.2f}")


Mean Squared Error: 11.11
Root Mean Squared Error: 3.33
Mean Absolute Error: 2.26
R-squared: 0.85


#### Guardado del modelo

In [22]:
from pickle import dump

dump(modelo_f, open("random_forest_regressor_42_sin_scal.sav", "wb"))

# De esta forma cargo el archivo
#from pickle import load
#with open("random_forest_regressor_42.sav", "rb") as f:
#    loaded_model = load(f)

In [23]:
X_testf.tail(10)

,CRIM,RM,DIS,LSTAT
131,1.19294,6.326,2.2710,12.26
346,0.06162,5.898,8.0136,12.67
365,4.55587,3.561,1.6132,7.12
132,0.59005,6.372,2.3274,11.12
371,9.23230,6.216,1.1691,9.53
412,18.81100,4.628,1.5539,34.37
436,14.42080,6.461,2.0026,18.05
411,14.05070,6.657,1.5275,21.22
86,0.05188,6.015,4.4272,12.86
75,0.09512,6.286,4.5026,8.94


In [26]:
boston_df.iloc[411, [0, 5, 7, 11,12]]

CRIM     14.0507
RM        6.6570
DIS       1.5275
LSTAT    21.2200
MEDV     17.2000
Name: 411, dtype: float64